# 2. Subsample Data Analysis

In [4]:
# Import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import psycopg2
import getpass
%matplotlib inline
plt.style.use('ggplot')

from sklearn.model_selection import train_test_split

In [5]:
# Database connection: information related to the previously created database
sqluser = 'postgres'
host = 'localhost'
dbname = 'mimic'
schema_name = 'mimiciii'

# Connect to postgres with a copy of the MIMIC-III database 
con = psycopg2.connect(dbname=dbname, user=sqluser, host=host, 
                      password=getpass.getpass(prompt='Password:'.format(sqluser)))

# State to the adequate Schema
cur = con.cursor()
cur.execute('SET search_path to {}'.format(schema_name))

Password:········


## Physiological Measurements and Lab Exams Data

### Hemodynamics data
Hemodynamic measurements are fundamental in terms of Heart Failure diagnosis and subtype determination. However, the main reference values are extracted from medical imaging, which are not available in this case. Thus, a first approach is to use the available hemodynamic data to model patient conditions with commonly used physiological measurements at the ICU. 

#### Systolic Heart Failure - Subsample Population
A subsample population of patients with Systolic Heart Failure is selected, and hemodynamic measurements are analyzed to obtain a DataFrame to model the data.

In [6]:
query = \
"""
SELECT *
FROM chartevents c
INNER JOIN diagnoses_icd d ON c.subject_id = d.subject_id
INNER JOIN d_items d_i ON c.itemid = d_i.itemid
WHERE d.icd9_code = '42820' AND d_i.category = 'Hemodynamics'
"""
df_hf_sys_hemo = pd.read_sql_query(query, con)
df_hf_sys_hemo

,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,...,row_id,itemid,label,abbreviation,dbsource,linksto,category,unitname,param_type,conceptid
0,181442,1767,147191,226932,220059,2168-04-14 21:00:00,2168-04-14 22:21:00,19786,32,32.0,...,12721,220059,Pulmonary Artery Pressure systolic,PAPs,metavision,chartevents,Hemodynamics,mmHg,Numeric,None
1,181443,1767,147191,226932,220060,2168-04-14 21:00:00,2168-04-14 22:21:00,19786,12,12.0,...,12722,220060,Pulmonary Artery Pressure diastolic,PAPd,metavision,chartevents,Hemodynamics,mmHg,Numeric,None
2,181444,1767,147191,226932,220061,2168-04-14 21:00:00,2168-04-14 22:21:00,19786,21,21.0,...,12723,220061,Pulmonary Artery Pressure mean,PAPm,metavision,chartevents,Hemodynamics,mmHg,Numeric,None
3,181445,1767,147191,226932,220074,2168-04-14 21:00:00,2168-04-14 22:21:00,19786,5,5.0,...,12729,220074,Central Venous Pressure,CVP,metavision,chartevents,Hemodynamics,mmHg,Numeric,None
4,181456,1767,147191,226932,220059,2168-04-14 22:00:00,2168-04-15 00:18:00,19786,34,34.0,...,12721,220059,Pulmonary Artery Pressure systolic,PAPs,metavision,chartevents,Hemodynamics,mmHg,Numeric,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11692,321247185,90680,193278,286464,227805,2180-08-29 22:00:00,2180-08-29 22:11:00,17950,4,NaN,...,14927,227805,SvO2 SQI,SvO2 SQI,metavision,chartevents,Hemodynamics,None,Text,None
11693,321247196,90680,193278,286464,227805,2180-08-29 23:00:00,2180-08-30 02:34:00,17950,2,NaN,...,14927,227805,SvO2 SQI,SvO2 SQI,metavision,chartevents,Hemodynamics,None,Text,None
11694,321247286,90680,193278,286464,227805,2180-08-30 01:00:00,2180-08-30 02:34:00,17950,4,NaN,...,14927,227805,SvO2 SQI,SvO2 SQI,metavision,chartevents,Hemodynamics,None,Text,None
11695,322343999,91603,170884,262288,227805,2195-10-13 20:10:00,2195-10-13 20:10:00,14891,1,NaN,...,14927,227805,SvO2 SQI,SvO2 SQI,metavision,chartevents,Hemodynamics,None,Text,None


In [7]:
# Remove all data that parameter type is 'Text'
df_hf_s_ = df_hf_sys_hemo.T.drop_duplicates().T
df_hf_s_ = df_hf_s_[df_hf_s_['param_type'] != 'Text']
df_hf_s_

,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,...,hadm_id,seq_num,icd9_code,row_id,label,abbreviation,dbsource,linksto,category,param_type
0,181442,1767,147191,226932,220059,2168-04-14 21:00:00,2168-04-14 22:21:00,19786,32,32,...,129892,2,42820,12721,Pulmonary Artery Pressure systolic,PAPs,metavision,chartevents,Hemodynamics,Numeric
1,181443,1767,147191,226932,220060,2168-04-14 21:00:00,2168-04-14 22:21:00,19786,12,12,...,129892,2,42820,12722,Pulmonary Artery Pressure diastolic,PAPd,metavision,chartevents,Hemodynamics,Numeric
2,181444,1767,147191,226932,220061,2168-04-14 21:00:00,2168-04-14 22:21:00,19786,21,21,...,129892,2,42820,12723,Pulmonary Artery Pressure mean,PAPm,metavision,chartevents,Hemodynamics,Numeric
3,181445,1767,147191,226932,220074,2168-04-14 21:00:00,2168-04-14 22:21:00,19786,5,5,...,129892,2,42820,12729,Central Venous Pressure,CVP,metavision,chartevents,Hemodynamics,Numeric
4,181456,1767,147191,226932,220059,2168-04-14 22:00:00,2168-04-15 00:18:00,19786,34,34,...,129892,2,42820,12721,Pulmonary Artery Pressure systolic,PAPs,metavision,chartevents,Hemodynamics,Numeric
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11658,18181260,63944,187913,249395,227547,2121-07-24 08:00:00,2121-07-24 08:23:00,20088,78,78,...,187913,8,42820,14790,SV (Arterial),SV (Arterial),metavision,chartevents,Hemodynamics,Numeric
11659,18180678,63944,187913,249395,227547,2121-07-23 16:00:00,2121-07-23 16:08:00,19783,75,75,...,187913,8,42820,14790,SV (Arterial),SV (Arterial),metavision,chartevents,Hemodynamics,Numeric
11660,18180702,63944,187913,249395,227543,2121-07-23 17:00:00,2121-07-23 17:23:00,19783,5.5,5.5,...,187913,8,42820,14788,CO (Arterial),CO (Arterial),metavision,chartevents,Hemodynamics,Numeric
11661,18180703,63944,187913,249395,227546,2121-07-23 17:00:00,2121-07-23 17:23:00,19783,14,14,...,187913,8,42820,14789,SVV (Arterial),SVV (Arterial),metavision,chartevents,Hemodynamics,Numeric


In [8]:
df_hf_s_.isna().sum()

row_id              0
subject_id          0
hadm_id             0
icustay_id          0
itemid              0
charttime           0
storetime           0
cgid                0
value               0
valuenum            0
valueuom            0
warning             0
error               0
resultstatus    11663
row_id              0
hadm_id             0
seq_num             0
icd9_code           0
row_id              0
label               0
abbreviation        0
dbsource            0
linksto             0
category            0
param_type          0
dtype: int64

In [9]:
df_hf_s_['value'].values.astype(str)
df_hf_s_['value'] = np.where(np.char.isnumeric(df_hf_s_['value'].values.astype(str)), df_hf_s_['value'].values, 'NaN')

In [10]:
cols = df_hf_s_['label'].unique()
rows = df_hf_s_['subject_id'].unique()
df_mean, df_max, df_min, df_std, df_col, df_id = [], [], [], [], [], []
df_hf_s_1 = pd.DataFrame(index=rows, columns=cols)

for ind in df_hf_s_1.index:
    df_s_1 = df_hf_s_[df_hf_s_['subject_id']==ind]
    for col in df_hf_s_1.columns:
        df_hf_s_1.loc[ind,col] = df_s_1[df_s_1['label']==col].value.astype('float64').mean()
        
df_hf_s_1

,Pulmonary Artery Pressure systolic,Pulmonary Artery Pressure diastolic,Pulmonary Artery Pressure mean,Central Venous Pressure,Cardiac Output (thermodilution),SvO2,PA Line cm Mark,Intra Cranial Pressure,Cardiac Output (CCO),ScvO2 (Presep),SVV (Arterial),SV (Arterial),CO (Arterial)
1767,37.0714,17.5714,26.3571,12.375,NaN,70.9216,48,NaN,4,NaN,NaN,NaN,NaN
2639,45.5,25.4048,33.6,16.6552,NaN,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3513,NaN,NaN,NaN,16.8289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3969,NaN,NaN,NaN,34.2919,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5244,NaN,NaN,NaN,83.1354,NaN,76.9615,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5727,NaN,NaN,NaN,17.4545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6901,NaN,NaN,NaN,9.95455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7449,37.2973,20.027,26.8333,12.3514,NaN,69,54.125,NaN,5,NaN,NaN,NaN,NaN
8498,NaN,NaN,NaN,13.875,NaN,NaN,NaN,NaN,NaN,64.7857,NaN,NaN,NaN
11242,NaN,NaN,NaN,6.42857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_hf_s_1.isna().sum()

Pulmonary Artery Pressure systolic     35
Pulmonary Artery Pressure diastolic    35
Pulmonary Artery Pressure mean         34
Central Venous Pressure                 1
Cardiac Output (thermodilution)        55
SvO2                                   39
PA Line cm Mark                        39
Intra Cranial Pressure                 54
Cardiac Output (CCO)                   45
ScvO2 (Presep)                         54
SVV (Arterial)                         52
SV (Arterial)                          52
CO (Arterial)                          52
dtype: int64

In [12]:
max_number_of_nas = 500
df_hf_s_2 = df_hf_s_1.loc[:, (df_hf_s_1.isnull().sum(axis=0) <= max_number_of_nas)]
df_hf_s_2

,Pulmonary Artery Pressure systolic,Pulmonary Artery Pressure diastolic,Pulmonary Artery Pressure mean,Central Venous Pressure,Cardiac Output (thermodilution),SvO2,PA Line cm Mark,Intra Cranial Pressure,Cardiac Output (CCO),ScvO2 (Presep),SVV (Arterial),SV (Arterial),CO (Arterial)
1767,37.0714,17.5714,26.3571,12.375,NaN,70.9216,48,NaN,4,NaN,NaN,NaN,NaN
2639,45.5,25.4048,33.6,16.6552,NaN,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3513,NaN,NaN,NaN,16.8289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3969,NaN,NaN,NaN,34.2919,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5244,NaN,NaN,NaN,83.1354,NaN,76.9615,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5727,NaN,NaN,NaN,17.4545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6901,NaN,NaN,NaN,9.95455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7449,37.2973,20.027,26.8333,12.3514,NaN,69,54.125,NaN,5,NaN,NaN,NaN,NaN
8498,NaN,NaN,NaN,13.875,NaN,NaN,NaN,NaN,NaN,64.7857,NaN,NaN,NaN
11242,NaN,NaN,NaN,6.42857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
max_number_of_nas = 35
df_hf_s_3 = df_hf_s_2.dropna(thresh=max_number_of_nas)
df_hf_s_3

,Pulmonary Artery Pressure systolic,Pulmonary Artery Pressure diastolic,Pulmonary Artery Pressure mean,Central Venous Pressure,Cardiac Output (thermodilution),SvO2,PA Line cm Mark,Intra Cranial Pressure,Cardiac Output (CCO),ScvO2 (Presep),SVV (Arterial),SV (Arterial),CO (Arterial)


In [14]:
max_number_of_nas = 25
df_hf_s_3 = df_hf_s_2.dropna(thresh=max_number_of_nas)
df_hf_s_3

,Pulmonary Artery Pressure systolic,Pulmonary Artery Pressure diastolic,Pulmonary Artery Pressure mean,Central Venous Pressure,Cardiac Output (thermodilution),SvO2,PA Line cm Mark,Intra Cranial Pressure,Cardiac Output (CCO),ScvO2 (Presep),SVV (Arterial),SV (Arterial),CO (Arterial)


In [15]:
max_number_of_nas = 15
df_hf_s_3 = df_hf_s_2.dropna(thresh=max_number_of_nas)
df_hf_s_3

,Pulmonary Artery Pressure systolic,Pulmonary Artery Pressure diastolic,Pulmonary Artery Pressure mean,Central Venous Pressure,Cardiac Output (thermodilution),SvO2,PA Line cm Mark,Intra Cranial Pressure,Cardiac Output (CCO),ScvO2 (Presep),SVV (Arterial),SV (Arterial),CO (Arterial)


In [16]:
max_number_of_nas = 5
df_hf_s_3 = df_hf_s_2.dropna(thresh=max_number_of_nas)
df_hf_s_3

,Pulmonary Artery Pressure systolic,Pulmonary Artery Pressure diastolic,Pulmonary Artery Pressure mean,Central Venous Pressure,Cardiac Output (thermodilution),SvO2,PA Line cm Mark,Intra Cranial Pressure,Cardiac Output (CCO),ScvO2 (Presep),SVV (Arterial),SV (Arterial),CO (Arterial)
1767,37.0714,17.5714,26.3571,12.375,NaN,70.9216,48,NaN,4,NaN,NaN,NaN,NaN
2639,45.5,25.4048,33.6,16.6552,NaN,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7449,37.2973,20.027,26.8333,12.3514,NaN,69,54.125,NaN,5,NaN,NaN,NaN,NaN
14230,38.0682,21.3409,28.25,14.0909,NaN,59.9737,44,NaN,3,NaN,NaN,NaN,NaN
28221,27.4483,16.3448,21.4828,19.0345,NaN,NaN,49,NaN,NaN,NaN,NaN,NaN,NaN
31994,44.7381,21.9048,30.8554,18.0633,NaN,69.1299,48,NaN,4.4,NaN,NaN,NaN,NaN
32447,37.0244,21.5157,28.2947,19.5199,NaN,63.9858,51.6,NaN,3.84848,NaN,NaN,NaN,NaN
55515,30.0968,15.2258,20.6452,11.0645,NaN,NaN,50,NaN,NaN,NaN,NaN,NaN,NaN
56201,40.9524,20.1905,28.4286,10.9524,NaN,64.2941,55,NaN,NaN,NaN,NaN,NaN,NaN
60969,47.9275,22.8551,32.5362,21.5797,NaN,NaN,51,NaN,NaN,NaN,NaN,NaN,NaN


The number of empty values, in this case NaNs, is high for this subsample. Only a threshold of 15 NaNs values removes all rows from the created DataFrame, making the modeling step difficult and especially inaccurate. Thus, it is necessary to include more information, considering not only hemodynamic data but further physiological measurements and lab exams data.

### Additional (Physiological) Data
From the MIMIC database, several additional physiological measurements and lab exams data is available. To improve the dataframe selection, relevant items were selected in addition to hemodynamics. Specifically, the categories arterial blood gas (ABG) data, hematology, chemistry, fluids, blood products/colloids, labs, cardiovascular, tandem heart and Non-invasive cardiac output monitor (NICOM).

#### Systolic Heart Failure - Subsample Population
Again, the subsample population of patients with Systolic Heart Failure is selected, and all selected physiological measurement data are analyzed in order to obtain a DataFrame to model the data.

In [17]:
query = \
"""
SELECT *
FROM chartevents c
INNER JOIN diagnoses_icd d ON c.subject_id = d.subject_id
INNER JOIN d_items d_i ON c.itemid = d_i.itemid
WHERE d.icd9_code = '42820' AND (d_i.category = 'Hemodynamics' OR d_i.category = 'ABG' OR d_i.category = 'Hematology' 
OR d_i.category = 'Chemistry' OR d_i.category = 'Fluids - Other (Not In Use)' OR d_i.category = 'Blood Products/Colloids' 
OR d_i.category = 'Labs' OR d_i.category = 'Cardiovascular (Pacer Data)' OR d_i.category = 'Tandem Heart' 
OR d_i.category = 'NICOM')
"""
df_hf_sys = pd.read_sql_query(query, con)
df_hf_sys

,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,...,row_id,itemid,label,abbreviation,dbsource,linksto,category,unitname,param_type,conceptid
0,3553129,28044,116475,265193.0,227466,2145-01-16 18:30:00,2145-01-16 19:33:00,20889,58.6,58.6,...,14827,227466,PTT,PTT,metavision,chartevents,Labs,None,Numeric with tag,None
1,3553128,28044,116475,265193.0,227442,2145-01-16 18:30:00,2145-01-16 19:22:00,20889,4.5,4.5,...,14756,227442,Potassium (serum),Potassium (serum),metavision,chartevents,Labs,None,Numeric with tag,None
2,3553127,28044,116475,265193.0,225677,2145-01-16 18:30:00,2145-01-16 19:22:00,20889,4.7,4.7,...,14119,225677,Phosphorous,Phosphorous,metavision,chartevents,Labs,None,Numeric,None
3,3553126,28044,116475,265193.0,225625,2145-01-16 18:30:00,2145-01-16 19:22:00,20889,9.1,9.1,...,14082,225625,Calcium non-ionized,Calcium non-ionized,metavision,chartevents,Labs,None,Numeric,None
4,3553124,28044,116475,265193.0,227467,2145-01-16 03:46:00,2145-01-16 05:45:00,20889,1.5,1.5,...,14828,227467,INR,INR,metavision,chartevents,Labs,None,Numeric with tag,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277220,90575581,10928,108955,220051.0,791,2114-12-27 03:00:00,2114-12-27 04:47:00,15331,.6,0.6,...,710,791,Creatinine (0-1.3),None,carevue,chartevents,Chemistry,None,None,None
277221,90575580,10928,108955,220051.0,788,2114-12-27 03:00:00,2114-12-27 04:47:00,15331,104,104.0,...,708,788,Chloride (100-112),None,carevue,chartevents,Chemistry,None,None,None
277222,90575579,10928,108955,220051.0,787,2114-12-27 03:00:00,2114-12-27 04:47:00,15331,28,28.0,...,707,787,Carbon Dioxide,None,carevue,chartevents,Chemistry,None,None,None
277223,90575578,10928,108955,220051.0,786,2114-12-27 03:00:00,2114-12-27 04:47:00,15331,8.6999998092651367,8.7,...,706,786,Calcium (8.4-10.2),None,carevue,chartevents,Chemistry,None,None,None


In [18]:
# Remove all data that parameter type is 'Text'
df_hf_sys_col = df_hf_sys.T.drop_duplicates().T
df_hf_sys_1 = df_hf_sys_col[df_hf_sys_col['param_type'] != 'Text']
df_hf_sys_1

,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,...,icd9_code,row_id,label,abbreviation,dbsource,linksto,category,unitname,param_type,conceptid
0,3553129,28044,116475,265193,227466,2145-01-16 18:30:00,2145-01-16 19:33:00,20889,58.6,58.6,...,42820,14827,PTT,PTT,metavision,chartevents,Labs,None,Numeric with tag,None
1,3553128,28044,116475,265193,227442,2145-01-16 18:30:00,2145-01-16 19:22:00,20889,4.5,4.5,...,42820,14756,Potassium (serum),Potassium (serum),metavision,chartevents,Labs,None,Numeric with tag,None
2,3553127,28044,116475,265193,225677,2145-01-16 18:30:00,2145-01-16 19:22:00,20889,4.7,4.7,...,42820,14119,Phosphorous,Phosphorous,metavision,chartevents,Labs,None,Numeric,None
3,3553126,28044,116475,265193,225625,2145-01-16 18:30:00,2145-01-16 19:22:00,20889,9.1,9.1,...,42820,14082,Calcium non-ionized,Calcium non-ionized,metavision,chartevents,Labs,None,Numeric,None
4,3553124,28044,116475,265193,227467,2145-01-16 03:46:00,2145-01-16 05:45:00,20889,1.5,1.5,...,42820,14828,INR,INR,metavision,chartevents,Labs,None,Numeric with tag,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277220,90575581,10928,108955,220051,791,2114-12-27 03:00:00,2114-12-27 04:47:00,15331,.6,0.6,...,42820,710,Creatinine (0-1.3),None,carevue,chartevents,Chemistry,None,None,None
277221,90575580,10928,108955,220051,788,2114-12-27 03:00:00,2114-12-27 04:47:00,15331,104,104,...,42820,708,Chloride (100-112),None,carevue,chartevents,Chemistry,None,None,None
277222,90575579,10928,108955,220051,787,2114-12-27 03:00:00,2114-12-27 04:47:00,15331,28,28,...,42820,707,Carbon Dioxide,None,carevue,chartevents,Chemistry,None,None,None
277223,90575578,10928,108955,220051,786,2114-12-27 03:00:00,2114-12-27 04:47:00,15331,8.6999998092651367,8.7,...,42820,706,Calcium (8.4-10.2),None,carevue,chartevents,Chemistry,None,None,None


In [19]:
df_hf_sys_1.isna().sum()

row_id               0
subject_id           0
hadm_id              0
icustay_id         162
itemid               0
charttime            0
storetime            0
cgid                 0
value              743
valuenum          1929
valueuom        158469
warning         196841
error           196841
resultstatus     78915
stopped          78882
row_id               0
hadm_id              0
seq_num              0
icd9_code            0
row_id               0
label                0
abbreviation    196841
dbsource             0
linksto              0
category             0
unitname        196841
param_type      196841
conceptid       275723
dtype: int64

In [20]:
df_hf_sys_1['value'].values.astype(str)
df_hf_sys_1['value'] = np.where(np.char.isnumeric(df_hf_sys_1['value'].values.astype(str)), df_hf_sys_1['value'].values, 'NaN')

C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [21]:
cols = df_hf_sys_1['label'].unique()
rows = df_hf_sys_1['subject_id'].unique()
df_mean, df_max, df_min, df_std, df_col, df_id = [], [], [], [], [], []
df_hf_s = pd.DataFrame(index=rows, columns=cols)

for ind in df_hf_s.index:
    df_s = df_hf_sys_1[df_hf_sys_1['subject_id']==ind]
    for col in df_hf_s.columns:
        df_hf_s.loc[ind,col] = df_s[df_s['label']==col].value.astype('float64').mean()
        
df_hf_s

,PTT,Potassium (serum),Phosphorous,Calcium non-ionized,INR,Prothrombin time,Platelet Count,HCO3 (serum),Anion gap,CK (CPK),...,Gentamicin (Trough),Intra Cranial Pressure,HCO3,SVI Change,Stroke Volume Index (SVI NICOM),CO / CI Change,Cardiac Index (CI NICOM),Stroke Volume Variation (SVV NICOM),Cardiac Output (CO NICOM),Stroke Volume (SV NICOM)
28044,61,4,4,9,NaN,NaN,217.889,34,13.5556,267,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92195,150,4.66667,NaN,9,2,13,167.2,26.3077,10.8462,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43632,NaN,4,2,8,NaN,NaN,256.5,19.7143,16.2857,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22518,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
530,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4799,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22165,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df_hf_s.isna().sum()

PTT                                    315
Potassium (serum)                      302
Phosphorous                            209
Calcium non-ionized                    319
INR                                    351
                                      ... 
CO / CI Change                         396
Cardiac Index (CI NICOM)               397
Stroke Volume Variation (SVV NICOM)    396
Cardiac Output (CO NICOM)              397
Stroke Volume (SV NICOM)               396
Length: 159, dtype: int64

In [23]:
max_number_of_nas = 500
df_hf_sys_2 = df_hf_s.loc[:, (df_hf_s.isnull().sum(axis=0) <= max_number_of_nas)]
df_hf_sys_2

,PTT,Potassium (serum),Phosphorous,Calcium non-ionized,INR,Prothrombin time,Platelet Count,HCO3 (serum),Anion gap,CK (CPK),...,Gentamicin (Trough),Intra Cranial Pressure,HCO3,SVI Change,Stroke Volume Index (SVI NICOM),CO / CI Change,Cardiac Index (CI NICOM),Stroke Volume Variation (SVV NICOM),Cardiac Output (CO NICOM),Stroke Volume (SV NICOM)
28044,61,4,4,9,NaN,NaN,217.889,34,13.5556,267,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92195,150,4.66667,NaN,9,2,13,167.2,26.3077,10.8462,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43632,NaN,4,2,8,NaN,NaN,256.5,19.7143,16.2857,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22518,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
530,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4799,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22165,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
max_number_of_nas = 35
df_hf_sys_3 = df_hf_sys_2.dropna(thresh=max_number_of_nas)
df_hf_sys_3

,PTT,Potassium (serum),Phosphorous,Calcium non-ionized,INR,Prothrombin time,Platelet Count,HCO3 (serum),Anion gap,CK (CPK),...,Gentamicin (Trough),Intra Cranial Pressure,HCO3,SVI Change,Stroke Volume Index (SVI NICOM),CO / CI Change,Cardiac Index (CI NICOM),Stroke Volume Variation (SVV NICOM),Cardiac Output (CO NICOM),Stroke Volume (SV NICOM)
28044,61,4,4,9,NaN,NaN,217.889,34,13.5556,267,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92195,150,4.66667,NaN,9,2,13,167.2,26.3077,10.8462,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18982,25.3333,4.28571,5.20833,7.85714,1.05263,14,246.605,23.2391,15.3261,231.333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31189,NaN,NaN,5,NaN,NaN,NaN,170.4,27,12,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87053,120,3.8,4.16667,8,2,14.5,35.0488,19.4545,22.5882,405.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30845,NaN,NaN,4.71429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17651,150,NaN,NaN,NaN,NaN,NaN,152.2,28.25,11.5,224.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30404,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
530,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Diastolic Heart Failure - Subsample Population
Complementary to the obtained dataframe, a subsample population of patients with Diastolic Heart Failure is selected, and all selected physiological measurement data are analyzed in order to obtain a DataFrame to model the data.

In [25]:
query = \
"""
SELECT *
FROM chartevents c
INNER JOIN diagnoses_icd d ON c.subject_id = d.subject_id
INNER JOIN d_items d_i ON c.itemid = d_i.itemid
WHERE d.icd9_code = '42830' AND (d_i.category = 'Hemodynamics' OR d_i.category = 'ABG' OR d_i.category = 'Hematology' 
OR d_i.category = 'Chemistry' OR d_i.category = 'Fluids - Other (Not In Use)' OR d_i.category = 'Blood Products/Colloids' 
OR d_i.category = 'Labs' OR d_i.category = 'Cardiovascular (Pacer Data)' OR d_i.category = 'Tandem Heart' 
OR d_i.category = 'NICOM')
"""
df_hf_dias = pd.read_sql_query(query, con)
df_hf_dias

,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,...,row_id,itemid,label,abbreviation,dbsource,linksto,category,unitname,param_type,conceptid
0,1939294,16560,109090,225372.0,227686,2113-02-01 09:31:00,2113-02-01 09:35:00,20889,88,88.0,...,15021,227686,Central Venous O2% Sat,Central Venous O2% Sat,metavision,chartevents,Labs,%,Numeric,None
1,1946362,16560,175933,269374.0,225664,2113-03-26 22:00:00,2113-03-26 22:13:00,19895,156,156.0,...,14111,225664,Glucose finger stick,Glucose finger stick,metavision,chartevents,Labs,None,Numeric,None
2,1941647,16560,109090,263449.0,225664,2113-02-10 18:00:00,2113-02-10 16:04:00,20622,194,194.0,...,14111,225664,Glucose finger stick,Glucose finger stick,metavision,chartevents,Labs,None,Numeric,None
3,1941640,16560,109090,263449.0,227442,2113-02-10 17:13:00,2113-02-10 18:25:00,20889,4.2,4.2,...,14756,227442,Potassium (serum),Potassium (serum),metavision,chartevents,Labs,None,Numeric with tag,None
4,1941639,16560,109090,263449.0,225677,2113-02-10 17:13:00,2113-02-10 18:25:00,20889,3.2,3.2,...,14119,225677,Phosphorous,Phosphorous,metavision,chartevents,Labs,None,Numeric,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637411,33717503,98744,168288,239780.0,220074,2127-06-11 23:00:00,2127-06-11 23:14:00,18522,10,10.0,...,12729,220074,Central Venous Pressure,CVP,metavision,chartevents,Hemodynamics,mmHg,Numeric,None
637412,33717496,98744,168288,239780.0,220074,2127-06-11 22:45:00,2127-06-11 22:54:00,18522,8,8.0,...,12729,220074,Central Venous Pressure,CVP,metavision,chartevents,Hemodynamics,mmHg,Numeric,None
637413,33717490,98744,168288,239780.0,220074,2127-06-11 22:15:00,2127-06-11 22:16:00,18522,11,11.0,...,12729,220074,Central Venous Pressure,CVP,metavision,chartevents,Hemodynamics,mmHg,Numeric,None
637414,33717483,98744,168288,239780.0,220074,2127-06-11 22:00:00,2127-06-11 22:15:00,18522,10,10.0,...,12729,220074,Central Venous Pressure,CVP,metavision,chartevents,Hemodynamics,mmHg,Numeric,None


In [26]:
df_hf_dias_col = df_hf_dias.T.drop_duplicates().T
df_hf_dias_1 = df_hf_dias_col[df_hf_dias_col['param_type'] != 'Text']
df_hf_dias_1

,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,...,icd9_code,row_id,label,abbreviation,dbsource,linksto,category,unitname,param_type,conceptid
0,1939294,16560,109090,225372,227686,2113-02-01 09:31:00,2113-02-01 09:35:00,20889,88,88,...,42830,15021,Central Venous O2% Sat,Central Venous O2% Sat,metavision,chartevents,Labs,%,Numeric,None
1,1946362,16560,175933,269374,225664,2113-03-26 22:00:00,2113-03-26 22:13:00,19895,156,156,...,42830,14111,Glucose finger stick,Glucose finger stick,metavision,chartevents,Labs,None,Numeric,None
2,1941647,16560,109090,263449,225664,2113-02-10 18:00:00,2113-02-10 16:04:00,20622,194,194,...,42830,14111,Glucose finger stick,Glucose finger stick,metavision,chartevents,Labs,None,Numeric,None
3,1941640,16560,109090,263449,227442,2113-02-10 17:13:00,2113-02-10 18:25:00,20889,4.2,4.2,...,42830,14756,Potassium (serum),Potassium (serum),metavision,chartevents,Labs,None,Numeric with tag,None
4,1941639,16560,109090,263449,225677,2113-02-10 17:13:00,2113-02-10 18:25:00,20889,3.2,3.2,...,42830,14119,Phosphorous,Phosphorous,metavision,chartevents,Labs,None,Numeric,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637411,33717503,98744,168288,239780,220074,2127-06-11 23:00:00,2127-06-11 23:14:00,18522,10,10,...,42830,12729,Central Venous Pressure,CVP,metavision,chartevents,Hemodynamics,mmHg,Numeric,None
637412,33717496,98744,168288,239780,220074,2127-06-11 22:45:00,2127-06-11 22:54:00,18522,8,8,...,42830,12729,Central Venous Pressure,CVP,metavision,chartevents,Hemodynamics,mmHg,Numeric,None
637413,33717490,98744,168288,239780,220074,2127-06-11 22:15:00,2127-06-11 22:16:00,18522,11,11,...,42830,12729,Central Venous Pressure,CVP,metavision,chartevents,Hemodynamics,mmHg,Numeric,None
637414,33717483,98744,168288,239780,220074,2127-06-11 22:00:00,2127-06-11 22:15:00,18522,10,10,...,42830,12729,Central Venous Pressure,CVP,metavision,chartevents,Hemodynamics,mmHg,Numeric,None


In [27]:
df_hf_dias_1.isna().sum()

row_id               0
subject_id           0
hadm_id              0
icustay_id         703
itemid               0
charttime            0
storetime            0
cgid                 0
value             1303
valuenum          3131
valueuom        364155
warning         454407
error           454407
resultstatus    181680
stopped         181543
row_id               0
hadm_id              0
seq_num              0
icd9_code            0
row_id               0
label                0
abbreviation    454407
dbsource             0
linksto              0
category             0
unitname        454407
param_type      454407
conceptid       635950
dtype: int64

In [28]:
df_hf_dias_1['value'].values.astype(str)
df_hf_dias_1['value'] = np.where(np.char.isnumeric(df_hf_dias_1['value'].values.astype(str)), df_hf_dias_1['value'].values, 'NaN')

C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [29]:
cols = df_hf_dias_1['label'].unique()
rows = df_hf_dias_1['subject_id'].unique()
df_mean, df_max, df_min, df_std, df_col, df_id = [], [], [], [], [], []
df_hf_d = pd.DataFrame(index=rows, columns=cols)

for ind in df_hf_d.index:
    df_dias = df_hf_dias_1[df_hf_dias_1['subject_id']==ind]
    for col in df_hf_d.columns:
        df_hf_d.loc[ind,col] = df_dias[df_dias['label']==col].value.astype('float64').mean()
        
df_hf_d

,Central Venous O2% Sat,Glucose finger stick,Potassium (serum),Phosphorous,PO2 (Mixed Venous),Ionized Calcium,INR,PTT,Prothrombin time,Platelet Count,...,Tobramycin (Trough),Tobramycin (Peak),Total Peripheral Resistance Index (TPRI) (NICOM),Total Peripheral Resistance (TPR) (NICOM),Thoracic Fluid Content (TFC) (NICOM),SVI Change,CO / CI Change,Gentamicin (Peak),Intra Cranial Pressure,Phenobarbital
16560,88,178.712,4,2.5,62.4,1,1,31,14.25,209.875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23734,74,125.474,4.33333,3.5,NaN,1,NaN,26.5,18,209.062,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1725,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3267,NaN,143.931,5,5.4,NaN,1,2,96.75,19,162.714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21021,NaN,121,5,3,NaN,NaN,NaN,NaN,NaN,298,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18123,NaN,NaN,NaN,3.66667,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21632,NaN,NaN,NaN,4.28571,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13599,NaN,NaN,4,4.5,NaN,NaN,NaN,32,NaN,342.067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
df_hf_d.isna().sum()

Central Venous O2% Sat    684
Glucose finger stick      491
Potassium (serum)         542
Phosphorous               326
PO2 (Mixed Venous)        663
                         ... 
SVI Change                703
CO / CI Change            703
Gentamicin (Peak)         703
Intra Cranial Pressure    702
Phenobarbital             702
Length: 168, dtype: int64

In [31]:
max_number_of_nas = 500
df_hf_dias_2 = df_hf_d.loc[:, (df_hf_d.isnull().sum(axis=0) <= max_number_of_nas)]
df_hf_dias_2

,Glucose finger stick,Phosphorous,Platelet Count,HCO3 (serum),Anion gap,BUN,Differential-Monos,Differential-Lymphs,Differential-Eos,Differential-Basos,...,Platelets,Glucose (70-105),Arterial PaO2,Arterial PaCO2,Arterial CO2(Calc),Fingerstick Glucose,Phosphorous(2.7-4.5),Magnesium (1.6-2.6),Differential-Polys,Calcium (8.4-10.2)
16560,178.712,2.5,209.875,29.4524,10.0714,23.8333,6,5,1.2,0,...,282.571,165.4,85.8,49.6,30,129,NaN,2,88,8
23734,125.474,3.5,209.062,25.0588,13.2353,16.9412,3,5,0,0,...,236,142.333,106.75,56,28,NaN,4,NaN,NaN,NaN
1725,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,139.5,151.706,119.656,39,23.9062,134.269,4,NaN,NaN,NaN
24223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,184.75,127,93.8333,87.75,38.6667,122,NaN,2,NaN,8
3267,143.931,5.4,162.714,20.1,20.6,30.5789,6.66667,15,0,0,...,198.222,116.882,NaN,NaN,NaN,125.973,4,2,70,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21021,121,3,298,24,11,22,8.66667,15.6667,0,0.333333,...,355.143,122.583,NaN,NaN,NaN,139.667,3,2,72,8
18123,NaN,3.66667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,355.868,119,103.365,46.0952,28.9683,132.023,3.66667,2,74,9
21632,NaN,4.28571,NaN,NaN,NaN,NaN,2.5,8.5,0,0,...,254.357,141.573,116.103,43.5981,26.1028,132.996,4.28571,2,77.5,8.125
13599,NaN,4.5,342.067,21.4375,11.6875,36.6875,3.55556,12.3333,6.16667,0,...,275.913,157.333,107.377,39.4762,22.9623,160.703,4.71429,2,62.4,9.11111


In [32]:
max_number_of_nas = 35
df_hf_dias_3 = df_hf_dias_2.dropna(thresh=max_number_of_nas)
df_hf_dias_3

,Glucose finger stick,Phosphorous,Platelet Count,HCO3 (serum),Anion gap,BUN,Differential-Monos,Differential-Lymphs,Differential-Eos,Differential-Basos,...,Platelets,Glucose (70-105),Arterial PaO2,Arterial PaCO2,Arterial CO2(Calc),Fingerstick Glucose,Phosphorous(2.7-4.5),Magnesium (1.6-2.6),Differential-Polys,Calcium (8.4-10.2)
16560,178.712,2.5,209.875,29.4524,10.0714,23.8333,6,5,1.2,0,...,282.571,165.4,85.8,49.6,30,129,NaN,2,88,8
23734,125.474,3.5,209.062,25.0588,13.2353,16.9412,3,5,0,0,...,236,142.333,106.75,56,28,NaN,4,NaN,NaN,NaN
3267,143.931,5.4,162.714,20.1,20.6,30.5789,6.66667,15,0,0,...,198.222,116.882,NaN,NaN,NaN,125.973,4,2,70,9
20815,NaN,3.66667,NaN,NaN,NaN,NaN,1.66667,1.33333,0,0,...,278.569,129.973,114.981,42.7358,25.9528,122.936,3.66667,2,61.3333,7.75
18982,162.736,5.20833,246.605,23.2391,15.3261,64.8478,1.4,6.65,0,0.05,...,264.495,135.696,136.343,46.9214,27.3741,150.099,5.0625,2,77.75,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20124,152.231,2.5,229.13,20.1379,20.9655,42.75,2.8,5.2,0.8,0,...,278.837,108.379,94.9268,45.7073,26.9756,121.728,3,2,70.5,7.66667
32522,NaN,2.66667,NaN,NaN,NaN,NaN,6,15,0,0,...,136.235,118.756,106.837,39.3953,22.1395,135.095,2.66667,2,73,10
21021,121,3,298,24,11,22,8.66667,15.6667,0,0.333333,...,355.143,122.583,NaN,NaN,NaN,139.667,3,2,72,8
21632,NaN,4.28571,NaN,NaN,NaN,NaN,2.5,8.5,0,0,...,254.357,141.573,116.103,43.5981,26.1028,132.996,4.28571,2,77.5,8.125


#### Systolic and Diastolic Heart Failure - DataFrame preparation
After processing and obtaining the dataframe for each subsample (Systolic and Diastolic Heart Failure), a unique DataFrame is created for modeling the data, experimenting if there is a model that can use the available data to separate patients with Systolic Heart Failure from patients with Diastolic Heart Failure.

In [33]:
hf_df = pd.merge(df_hf_sys_3, df_hf_dias_3)
hf_df

,PTT,Potassium (serum),Phosphorous,Calcium non-ionized,INR,Prothrombin time,Platelet Count,HCO3 (serum),Anion gap,CK (CPK),...,Gentamicin (Trough),Intra Cranial Pressure,HCO3,SVI Change,Stroke Volume Index (SVI NICOM),CO / CI Change,Cardiac Index (CI NICOM),Stroke Volume Variation (SVV NICOM),Cardiac Output (CO NICOM),Stroke Volume (SV NICOM)
0,25.3333,4.28571,5.20833,7.85714,1.05263,14,246.605,23.2391,15.3261,231.333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,4,1.57143,9,NaN,NaN,260.5,13.8,14.6842,134,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,119,4.5,4.5,9,2,NaN,265.4,24.9545,17.9318,607,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,4.2,NaN,4,NaN,105,34.5,11,103.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,67.8,4.05882,3.03704,8.05263,3.16667,16.4286,35.6449,23.4968,14.6753,428.429,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,5.72727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
df_hf_dias_3.loc[:, 'HF_diagnosis'] = 'Dias'
hf_df_1 = pd.concat([df_hf_sys_3, df_hf_dias_3], axis=1)
hf_df_1['HF_diagnosis'] = hf_df_1.loc[:, 'HF_diagnosis'].fillna(value='Sys')
hf_df_1

C:\Users\PC\Anaconda3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\PC\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,PTT,Potassium (serum),Phosphorous,Calcium non-ionized,INR,Prothrombin time,Platelet Count,HCO3 (serum),Anion gap,CK (CPK),...,Glucose (70-105),Arterial PaO2,Arterial PaCO2,Arterial CO2(Calc),Fingerstick Glucose,Phosphorous(2.7-4.5),Magnesium (1.6-2.6),Differential-Polys,Calcium (8.4-10.2),HF_diagnosis
68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sys
148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,173.167,103.795,41.9744,27.6154,151.239,3,2,70.7143,7.66667,Dias
305,NaN,NaN,3.66667,9,NaN,NaN,329.5,25.5,17,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sys
353,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sys
530,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sys
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87754,36,4,3,9,NaN,17,106.692,28.1818,12.7273,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sys
89008,35,4,NaN,NaN,NaN,17.5,97,26,10.3333,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sys
90680,NaN,4,2,8,NaN,NaN,160,27.3333,12.5556,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sys
91603,NaN,4,NaN,NaN,NaN,NaN,106.333,27.5714,10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sys


In [35]:
# Save final DataFrame with all curated data from selected subsample
hf_df_1.to_pickle('HF_df.pkl')

## Conclusions
A fundamental step to analyze the MIMIC database in the context of Heart Failure patients is to know the related available data. Making use of the lab exams and physiological measurements done at the ICU, it could be interesting to determine if there is relevant information about Heart Failure patients and specific differences between Heart Failure subtypes. Heart Failure diagnosis is done by hemodynamic analysis of medical imaging and extracted parameters. Thus, the first measurements taken for the analysis are the ones indicated with the tag "Hemodynamics". A DataFrame of Systolic Failure patients is prepared but the limited number of patients with values forces to include other measurements such as arterial blood gas (ABG) data, hematology, chemistry, fluids, blood products/colloids, labs, cardiovascular, tandem heart and non-invasive cardiac output monitor (NICOM). Then, a subsample of patients with Systolic and Diastolic Failure is selected, prepared and cleaned to build a DataFrame, including a label with the correspondent diagnosis, for modeling the data, experimenting if there is a model that can use the available data to identify and differentiate patients with Systolic Heart Failure and with Diastolic Heart Failure. The DataFrame is saved as a pickle file to be used in Part 3: Data Modeling.